In [27]:
import sys
import os
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K
from tensorflow.python.framework.ops import disable_eager_execution
from keras.applications import VGG16


In [14]:
model = VGG16(weights='imagenet', include_top=False)


58892288/58889256 [==============================] - 20s 0us/step


In [16]:
model.summary()


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [18]:
cnn=Sequential()
for capa in model.layers:
    cnn.add(capa)

In [19]:
cnn.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, None, None, 256)  

In [20]:
for layer in cnn.layers:
    layer.trainable=False

In [21]:
cnn.add(Dense(6,activation='softmax'))


In [22]:
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, None, None, 256)  

In [36]:
def create_model():
    model=applications.vgg16.VGG16()
    cnn=Sequential()
    for capa in model.layers:
        cnn.add(capa)
    cnn.layers.pop()
    for layer in cnn.layers:
        layer.trainable=False
    cnn.add(Dense(6,activation='softmax'))
    
    return cnn

In [37]:
disable_eager_execution()


K.clear_session()



data_entrenamiento = './data/Train'
data_validacion = './data/Validation'

"""
Parameters
"""
epocas=10
longitud, altura = 224, 224
batch_size = 32
pasos = 200
validation_steps = 200
filtrosConv1 = 32
filtrosConv2 = 64
tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2)
clases = 6
lr = 0.0005


In [38]:
entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

print("Indices: ")
print(entrenamiento_generador.class_indices)


validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

Found 192 images belonging to 6 classes.
Indices: 
{'Alfil': 0, 'Caballo': 1, 'Dama': 2, 'Peon': 3, 'Rey': 4, 'Torre': 5}
Found 191 images belonging to 6 classes.


In [39]:
cnn = create_model()

In [40]:
cnn.compile(loss='categorical_crossentropy',
            optimizer=optimizers.Adam(lr=lr),
            metrics=['accuracy'])

In [ ]:
cnn.fit(
    entrenamiento_generador,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps)




target_dir = './conocimiento/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
cnn.save('./conocimiento/modelo.h5')
cnn.save_weights('./conocimiento/pesos.h5')

print("Indices: ")
print(entrenamiento_generador.class_indices)

Epoch 1/10
200/200 [==============================] - 9847s 49s/step - loss: 1.7872 - accuracy: 0.1978 - val_loss: 1.7866 - val_accuracy: 0.1784
Epoch 2/10
200/200 [==============================] - 9484s 47s/step - loss: 1.7789 - accuracy: 0.2036 - val_loss: 1.7824 - val_accuracy: 0.1789
Epoch 3/10
200/200 [==============================] - ETA: 0s - loss: 1.7728 - accuracy: 0.2023 